# TFM

## Tabla de contenidos

* [1. Objetivo](#cap1)
    * [1.1. Participantes](#section_1_1)
* [2. Librerías y carga de datos](#cap2)
* [3. Feature engineering](#cap3)
* [4. EDA](#cap4)
* [5. Análisis sentimientos titulo](#cap5)
    * [Section 1.2](sSection_1_2)
        * [Section 1.2.1](#section_1_2_1)
        * [Section 1.2.2](#section_1_2_2)
        * [Section 1.2.3](#section_1_2_3)
* [6. Análisis sentimientos post](#cap6)
* [7. Conclusiones](#cap7)

## 1. Objetivo <a class="anchor" id="cap1"></a>

Este proyecto tiene como propósito que los estudiantes identifiquen un problema abierto en el área del análisis de datos apoyado por técnicas de Inteligencia Artificial en el ámbito de su preferencia (Educación, ambiente, criminología, gobierno, etc.). Una vez identificado el problema el estudiante aplicará los conocimientos y competencias, vistos y adquiridos en el Master para hacer un tratamiento de datos al nivel que escoja (Descriptivo, diagnóstico, predictivo, prescriptivo).

### 1.1. Participantes <a class="anchor" id="section_1_1"></a>

* Mireya Harillo Gámez
* Óscar Tienda Beteta
* Henry Ocaña Luna
* David de la Torre Amengual

## 2. Librerías y carga de datos <a class="anchor" id="cap2"></a>

In [1]:
import pandas as pd
import seaborn as sns

A continuación cargamos los datos de reddit con los que vamos a trabajar y mostramos una porción.

In [2]:
#reddit_raw_dataset = pd.read_csv('reddit_wsb.csv')
reddit_raw_dataset = pd.read_csv('https://github.com/david-dlta/recursosTFM/reddit_wsb.csv')
#raw_dataset = r_data[pd.to_datetime(r_data.timestamp).dt.year>=2021]
reddit_raw_dataset.head()

HTTPError: HTTP Error 404: Not Found

Hacemos lo mismo con los datos de los precios de las acciones de GameStop Corp. (GME).

In [ ]:
gme_raw_dataset = pd.read_csv('GME_stock.csv')
gme_raw_dataset.head()

## 3. Feature engineering <a class="anchor" id="cap3"></a>

Los datos con los que contamos de partida en el dataset no se ajustan totalmente a las nuestras necesidades. Por ello debemos realizar algunas transformaciones sobre estos para poder llevar a cabo el proyecto.

Creamos tres atributos nuevos partiendo del atributo *timestamp*.

In [ ]:
dataset = reddit_raw_dataset.copy()

dataset['year'] = pd.to_datetime(reddit_raw_dataset['timestamp']).dt.year
dataset['month'] = pd.to_datetime(reddit_raw_dataset['timestamp']).dt.month
dataset['day'] = pd.to_datetime(reddit_raw_dataset['timestamp']).dt.day

dataset

## EDA <a class="anchor" id="cap4"></a>

In [ ]:
reddit_raw_dataset.info()

Primero vamos a extraer información descriptiva de los datos. Con el parámetro *include = 'all'* especificamos que queremos información de todos los atributos del dataset, incluso si no son numéricos.

In [ ]:
reddit_raw_dataset.describe(include='all')

**Observaciones**

In [ ]:
reddit_raw_dataset.head()

## 5. Análisis sentimiento títulos <a class="anchor" id="cap5"></a>

## 6. Análisis sentimiento posts <a class="anchor" id="cap6"></a>

## 7. Conclusiones <a class="anchor" id="cap7"></a>